In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import string, sys, tflearn

# Query/Ad length
query_len = 35
ad_len = 140

# char dictionary
gap_fill_char = '\xff'
unknown_char = '\xfe'
ad_col_seperator_1 = '\xfd'
ad_col_seperator_2 = '\xfc'

char_set = set(string.printable) - set(string.ascii_uppercase)
char_set.add(unknown_char); 
char_set.add(ad_col_seperator_1); 
char_set.add(ad_col_seperator_2)
char_set_len = len(char_set)

char_to_ix = {}; ix_to_char = {};
for i,ch in enumerate(char_set):
    char_to_ix[ch] = i
    ix_to_char[i] = ch

# Filler
def fill_string(s, length, char='\xff'):
    length_of_string = len(s)
    if length_of_string < length:
        return s + (length - length_of_string)*char
    else:
        return s[:length]

# Load editorial data set
def load_editorial_data(dataset, nrows=None):

    data_df = pd.read_csv(dataset, delimiter='\t', nrows=nrows)
    data_df.columns = ['phrase_canon_user_query', 'ad_title', 'ad_description', 'ad_url','click']
    data_df['ad'] = data_df['ad_title'] + ad_col_seperator_1 + data_df['ad_description'] + ad_col_seperator_2 + data_df['ad_url']
    data_df = data_df[['phrase_canon_user_query', 'ad', 'click']]
    data_df['phrase_canon_user_query'] = data_df['phrase_canon_user_query'].str.slice(start=0,stop=query_len).astype(str)
    data_df['ad'] = data_df['ad'].str.slice(start=0,stop=ad_len).astype(str)
    query = data_df['phrase_canon_user_query'].apply(fill_string, length=query_len).values.tolist()
    ad = data_df['ad'].apply(fill_string, length=ad_len).values.tolist()
    labels = data_df['click'].values.astype(np.float32)
    del data_df
    return [ad, query, labels]

# Vectorize query/ad text (1-hot encoding)
def vectorize_text(in_list):

    out_tensor = np.zeros((len(in_list), len(in_list[0]), char_set_len), dtype=np.float32)
    for i, s in enumerate(in_list):
        for t,char in enumerate(s):
            if (char in char_set):
                out_tensor[i, t, char_to_ix[char]] = 1

            elif (char != gap_fill_char):
                out_tensor[i, t, char_to_ix[unknown_char]] = 1

    return out_tensor

# Convolutional Block
def conv_block(x, filter_size, number_of_filters, num_of_layers, trainable, restore, name="Conv1D"):
    net = x
    for i in range(num_of_layers):
        net = tflearn.layers.conv.conv_1d(net, nb_filter=number_of_filters, 
                filter_size=filter_size, strides=1, 
                padding='same', activation= 'linear', 
                weights_init = 'variance_scaling',
                trainable=trainable, restore=restore, name=name)

        net = tflearn.layers.normalization.batch_normalization(
                net, trainable=trainable, restore=restore, 
                name="batch_normalization/"+name)

        net = tf.nn.relu(net, name="ReLU/"+name)
    return net


# Model - Ads
baseFactor = 64
ad_place_holder = tf.placeholder(tf.float32, shape=[None, ad_len, char_set_len])
ad_conv_1 = tflearn.layers.conv.conv_1d(ad_place_holder, nb_filter=baseFactor, 
        filter_size=3, strides=1, padding='same', 
        activation= 'linear', weights_init = 'variance_scaling',
        trainable=True, restore=True, name="AdConv1")

ad_conv_2 = conv_block(ad_conv_1, filter_size=3, number_of_filters=baseFactor,
        num_of_layers=2, trainable=True, restore=True, name="AdConv2")
ad_conv_3 = conv_block(ad_conv_2, filter_size=3, number_of_filters=baseFactor, 
        num_of_layers=2, trainable=True, restore=True, name="AdConv3")
ad_conv_3_max_pool = tflearn.layers.conv.max_pool_1d(ad_conv_3, 
        kernel_size=2, name="AdConv3MaxPool")

# Model - Queries
query_place_holder = tf.placeholder(tf.float32, shape=[None, query_len, char_set_len])
query_conv_1 = tflearn.layers.conv.conv_1d(query_place_holder, nb_filter=baseFactor, 
        filter_size=3, strides=1, 
        padding='same', activation= 'linear', weights_init = 'variance_scaling',
        trainable=True, restore=True, name="QueryConv1")

query_conv_2 = conv_block(query_conv_1, filter_size=3, number_of_filters=baseFactor, 
        num_of_layers=2, trainable=True, restore=True, name="QueryConv2")
query_conv_3 = conv_block(query_conv_2, filter_size=3, number_of_filters=baseFactor, 
        num_of_layers=2, trainable=True, restore=True, name="QueryConv3")

# Ad/Query Cross Convolutional matrix
seq_length_of_query = query_conv_3.get_shape()[1].value
seq_length_of_ad = ad_conv_3_max_pool.get_shape()[1].value

ad_entended_matrix = []
for i in range(seq_length_of_query):
    ad_entended_matrix.append(ad_conv_3_max_pool)
ad_entended_matrix = tf.concat(ad_entended_matrix, axis=1)

query_extended_matrix = []
queryMatrixUnpacked = tf.unstack(query_conv_3, num=None, axis=1)
for actualQueryRow in queryMatrixUnpacked:
    for i in range(seq_length_of_ad):
        query_extended_matrix.append(actualQueryRow)
query_extended_matrix = tf.stack(query_extended_matrix, axis=1)
query_ads = tf.concat([query_extended_matrix, ad_entended_matrix], axis=2)

# Session run
sess = tf.Session()
sess.run(tf.global_variables_initializer())
writer = tf.summary.FileWriter("./run_1", graph=tf.get_default_graph())

# Testing
[ads, queries, labels] = load_editorial_data("small_train.txt")
ad_vectors =  vectorize_text(ads[:512]) 
# Input
query_vectors =  vectorize_text(queries[:512]) 
labels_vector = labels[:512]

ads_ = sess.run(ad_conv_3_max_pool, feed_dict={ad_place_holder:ad_vectors})
queries_ = sess.run(query_conv_3, feed_dict={ad_place_holder:ad_vectors, query_place_holder:query_vectors})
ad_matrix = sess.run(ad_entended_matrix, feed_dict={ad_place_holder:ad_vectors, query_place_holder:query_vectors})
query_matrix = sess.run(query_extended_matrix, feed_dict={ad_place_holder:ad_vectors, query_place_holder:query_vectors})
query_ads_ = sess.run(query_ads, feed_dict={ad_place_holder:ad_vectors, query_place_holder:query_vectors})


